In [1]:
import requests
import pandas as pd

In [2]:
## Base path the active DataARC API
BASE_URL = "https://api.data-arc.org"

## A basic function to generate a path based on the
## documentation located at https://api.data-arc.org/documentation
## example: MAKE_PATH("datasets")
MAKE_PATH = lambda path: f"{BASE_URL}/{path}"

# End point URIs
DATASET_END_POINT = "datasets"
DATASET_FEATURES_END_POINT = "features"

# Utility Functions

def get_dataset_by_name(name: str) -> dict:
    """
    Simple method for quickly querying and filtering a dataset
    based on a name

    params
    ------
    name: str
      The string name as it is in the database
    
    returns
    -------
    dict
      The returned JSON data from the API
    """
    datasets = requests.get(MAKE_PATH(DATASET_END_POINT)).json()
    dataset = list(filter(lambda d: d.get('name') == name, datasets))[0]
    return dataset


def map_feature_to_nested_property(feature, prop='properties'):
    """
    This is a basic function to extract nested dictionaries from features
    returned from the DataARC database. This is not the same as flattening
    as only the specified nested dictionary is extracted. 
    The original feature id as kept as `_id` in case future REST queries
    require this database specific id to be in place

    params
    ------
    feature: dict
      the feature returned from the DataARC API
    
    prop: str
      the nested property to be returned - usually a dictionary value

    returns
    -------
    dict
      the extracted nested property with the _id in place
    """
    properties = feature.get(prop, {})
    ## Capture the database ID to run additional feature queries
    properties['_id'] = feature.get('id') 
    return properties


def get_combinators_from_features(features: list) -> dict:
    """
    Collect combinators based on a unique list of feature _id values

    params
    ------
    features: List[str]
      a unique list of str _id values resulted from the /features api
    
    returns
    -------
    dict
      a dictionary containing both the found related and contextual unique
      ids as properties
    """
    # Use the original feature _id to collect concepts through their combinators
    data = requests.get(MAKE_PATH("combinators"), {"features_in": features}).json()
    # >> List[List[Dict]]
    # Combinators using the `_in` suffix will return a list of lists with dictionaries
    combinator_collection = [combinator.get('concepts') for combinator in data]

    related = []
    contextual = []

    for combinators in combinator_collection:
        for combinator in combinators:
            related += combinator.get('related')
            contextual += combinator.get('contextual')
    
    return {
        'related': set(related),
        'contextual': set(contextual)
    }


def make_concepts_request(concept_ids: list) -> pd.DataFrame:
    """
    Collect concepts from their _id values

    params
    ------
    concept_ids: List[str]
      The id returned from the API end point for /concepts
    
    returns
    -------
    pd.DataFrame
      A precompiled DataFrame with the concept results
    """
    data = requests.get(MAKE_PATH("concepts"), {
        "id_in": concept_ids
    }).json()
    return pd.DataFrame(data)


def get_features_by_dataset_id(dataset_id: str) -> pd.DataFrame:
    """
    Given a dataset id (_id), construct a DataFrame with the results

    params
    ------
    dataset_id: str
      the dataset id (or _id) returned from the /datasets endpoint
    
    returns
    -------
    pd.DataFrame
      the constructed DataFrame
    """
    params = {
        "dataset": dataset_id,
        "_limit": -1 ## This will retrieve all records without a limit
    }

    features_data = requests.get(MAKE_PATH(DATASET_FEATURES_END_POINT), params=params).json()

    return pd.DataFrame(map(map_feature_to_nested_property, features_data))



### Use case 1: 
I'm interested in different communities' reliance on marine vs. terrestrial resources for food. I want to find places where there's a fairly balanced reliance on these. As a ballpark figure, I'd start by looking at places where the ration is something between 40/60 and 60/40 marine to terrestrial.

Step 1: I know from exploring on the front end UI that nabonosead has some of the data I want... I've seen indicators of both marine and terrestrial resources in that data when I searched on a concept about 'livestock'. I head to the 'advanced user interface' aka this jupyter notebook

Step 2: I make an api call to get the list of datasets (so I know what the nabonosead dataset is called in the API) and another API call to get the list of field names in the nabonosead data

In [3]:
# Collect the nabonosead dataset dictionary
nabonosead_dataset = get_dataset_by_name('nabonosead')
# The dataset has all fields in the initial return - this can be extracted from the dict
nabonosead_dataset_fields = nabonosead_dataset.get('fields')
# Create a DataFrame from the extracted fields
nabonosead_fields_df = pd.DataFrame(nabonosead_dataset_fields)
nabonosead_fields_df

,type,review,missing,_id,name,source,parent,title,createdAt,updatedAt,__v,dataset,citation,description,url,id
0,number,True,False,5fc9e3007c39e36162afc6e4,id,id,None,Id,2020-12-04T07:19:28.369Z,2021-01-05T22:01:03.545Z,0,5f41cae4b2e7238640808184,None,The unique SEAD identification number.,None,5fc9e3007c39e36162afc6e4
1,number,True,False,5fc9e3007c39e36162afc6e8,sampledata_sample_name,sampleData --> sample_name,None,Sampledata Sample Name,2020-12-04T07:19:28.369Z,2021-01-05T22:01:16.025Z,0,5f41cae4b2e7238640808184,None,Name of the soil sample from which the data ar...,None,5fc9e3007c39e36162afc6e8
2,number,True,False,5fc9e3007c39e36162afc6e6,sampledata_site_id,sampleData --> site_id,None,Sampledata Site Id,2020-12-04T07:19:28.369Z,2021-01-05T22:01:34.995Z,0,5f41cae4b2e7238640808184,None,The unique number assigned to a particular col...,None,5fc9e3007c39e36162afc6e6
3,string,True,False,5fc9e3007c39e36162afc6ea,sampledata_dating_type,sampleData --> dating_type,None,Sampledata Dating Type,2020-12-04T07:19:28.369Z,2021-01-05T22:01:47.378Z,0,5f41cae4b2e7238640808184,None,Broad dating method type used to date sample.,None,5fc9e3007c39e36162afc6ea
4,number,True,False,5fc9e3007c39e36162afc6e9,sampledata_sample_group_id,sampleData --> sample_group_id,None,Sampledata Sample Group Id,2020-12-04T07:19:28.369Z,2021-01-05T22:01:59.783Z,0,5f41cae4b2e7238640808184,None,Identification number for the sample group.,None,5fc9e3007c39e36162afc6e9
5,begin,True,False,5fc9e3007c39e36162afc6eb,sampledata_start,sampleData --> start,None,Sampledata Start,2020-12-04T07:19:28.369Z,2021-01-05T22:02:08.508Z,0,5f41cae4b2e7238640808184,None,Earliest possible date specimens were deposited.,None,5fc9e3007c39e36162afc6eb
6,string,True,False,5fc9e3007c39e36162afc6e7,sampledata_site_name,sampleData --> site_name,None,Sampledata Site Name,2020-12-04T07:19:28.369Z,2021-01-05T22:02:15.698Z,0,5f41cae4b2e7238640808184,None,Name for the collection site based on geograph...,None,5fc9e3007c39e36162afc6e7
7,string,True,False,5fc9e3007c39e36162afc6e5,country,country,None,Country,2020-12-04T07:19:28.369Z,2021-01-05T22:02:25.828Z,0,5f41cae4b2e7238640808184,None,The country in which the sample(s) were taken.,None,5fc9e3007c39e36162afc6e5
8,begin,True,False,5fc9e3007c39e36162afc6ec,sampledata_end,sampleData --> end,None,Sampledata End,2020-12-04T07:19:28.369Z,2021-01-05T22:02:33.309Z,0,5f41cae4b2e7238640808184,None,Latest possible date specimens were deposited.,None,5fc9e3007c39e36162afc6ec
9,number,True,False,5fc9e3007c39e36162afc6f0,indicators_wild,indicators --> wild,None,Indicators Wild,2020-12-04T07:19:28.369Z,2021-01-05T22:02:41.598Z,0,5f41cae4b2e7238640808184,None,Indicates that the specimen is from a wild spe...,None,5fc9e3007c39e36162afc6f0


Step 3: I can see that Indicators Freshwater Fish, Indicators Marine Fish, Indicators Domestic and Indicators Wild seem like likely candidate fields to point at sites I'd want to investigate.

Step 4: I want to construct my query to get sites with a balanced ratio... So I start by trying an API query that gets all the sites where Indicators Freshwater Fish OR Indicators Marine Fish OR Indicators Domestic OR Indicators Wild >0 and put them into a dataframe.

In [4]:
# Use the dataset id from the above dictionary to collect the features
# For this we can call our handy function from above
features_df = get_features_by_dataset_id(nabonosead_dataset.get('id'))

# Sub-select DataFrame with our conditional OR '|' operations
sub_features = features_df[
    (features_df['indicators_wild'] > 0) | 
    (features_df['indicators_marine_fish'] > 0) | 
    (features_df['indicators_domestic'] > 0) | 
    (features_df['indicators_freshwater_fish'] > 0)
]

Step 5: That's given me more sites than what I really want (balanced and unbalanced ratios are included). But I can filter the dataframe using the magic of pandas. I do 

```
df.mydataset = df.loc[
    (Indicators Freshwater Fish + Indicators Marine Fish) / (Indicators Domestic + Indicators Wild) > 40 & 
    (Indicators Freshwater Fish + Indicators Marine Fish) / (Indicators Domestic + Indicators Wild) < 60
]
``` 


In [5]:
# Create a new column in a new DataFrame with the `ratio` applied
features_with_ratio_df = sub_features.assign(
    ratio=(sub_features['indicators_freshwater_fish'] + sub_features['indicators_marine_fish']) / \
          (sub_features['indicators_domestic'] + sub_features['indicators_wild'])
)
# Drop any NaN values resulting from division by 0
features_with_a_valid_ratio = features_with_ratio_df.dropna(subset=["ratio"])

# Collect the balanced features remaining
balanced = features_with_a_valid_ratio[
    (features_with_a_valid_ratio['ratio'] > 0.4) & (features_with_a_valid_ratio['ratio'] < 0.6)
]
balanced

,id,country,sampledata_site_id,sampledata_site_name,sampledata_sample_name,sampledata_sample_group_id,sampledata_dating_type,sampledata_start,sampledata_end,sampledata_age_name,sampledata_age_abbreviation,indicators_domestic,indicators_wild,indicators_marine_mammal,indicators_marine_fish,_id,indicators_freshwater_fish,ratio
344,8081,Iceland,1293,Hofstaðir,1066,1455,Relative dates,1050.0,1055.0,III,HOF_Phase_III,42.0,179.0,NaN,2.0,5ff3a41324b939001c694831,94.0,0.434389
369,8106,Iceland,1293,Hofstaðir,1083,1456,Relative dates,1050.0,1055.0,III,HOF_Phase_III,30.0,278.0,NaN,10.0,5ff3a41324b939001c69484a,116.0,0.409091
387,8124,Iceland,1293,Hofstaðir,4,1459,Relative dates,1000.0,1050.0,II,HOF_Phase_II,745.0,7432.0,1.0,800.0,5ff3a41324b939001c69485c,4034.0,0.591170
391,8128,Iceland,1293,Hofstaðir,5b,1459,Relative dates,1000.0,1050.0,II,HOF_Phase_II,193.0,1629.0,NaN,241.0,5ff3a41324b939001c694860,801.0,0.571899
395,8132,Iceland,1293,Hofstaðir,6b,1459,Relative dates,930.0,1050.0,I-II,HOF_Phase_I-II,78.0,420.0,NaN,71.0,5ff3a41324b939001c694864,196.0,0.536145
401,8138,Iceland,1293,Hofstaðir,6hk,1459,Relative dates,930.0,1000.0,I,HOF_Phase_I,607.0,2734.0,2.0,255.0,5ff3a41324b939001c69486a,1498.0,0.524693
405,8142,Iceland,1293,Hofstaðir,6n,1459,Relative dates,930.0,1000.0,I,HOF_Phase_I,239.0,2227.0,NaN,680.0,5ff3a41324b939001c69486e,792.0,0.596918
414,8151,Iceland,1293,Hofstaðir,7A,1459,Relative dates,930.0,1000.0,I,HOF_Phase_I,64.0,264.0,1.0,6.0,5ff3a41324b939001c694877,173.0,0.545732
900,8637,Iceland,1306,Selhagi,5,1489,Relative dates,1170.0,1477.0,Late,SLH_3,8.0,104.0,NaN,8.0,5ff3a41324b939001c694a5d,42.0,0.446429


Step 6: I am happy and maybe make myself some nice charts showing the different ratios in my subset of sites with balanced ratios...

In [6]:
f"{len(balanced)} features found"

'9 features found'

Step 7: I wonder what other data might be connected via concepts and also relevant to these sites with balanced resource use. I run an API call against the ids of these items from nabonosead to get the set of concepts that are 'related' and 'contextual' to their concepts <-- this part I'm not sure how you have it set up in the API.

In [7]:
# Collect the unique list of balanced feature _id values for the API
unique_feature_ids = set(balanced['_id'])

# To get concepts, we must first travel through the combinators
# Collect combinators
combinators = get_combinators_from_features(unique_feature_ids)

# For this we want the `related` and `contextual` combinators
related = combinators.get('related')
contextual = combinators.get('contextual')

In [8]:
# Use the combinator ids to collect concepts using another handy function
related_df = make_concepts_request(related)
contextual_df = make_concepts_request(contextual)

Step 8: I could go back to the UI and search on these concepts, or I could search for the datasets linked to one of the related concepts from here...

### Use case: 
I'm interested in looking at places with lots of data from short well defined timespans. From looking around in the UI, I can see the tephrabase data should let me find places with well defined chronologies and then I could see what other data is available in these places and how much of it there is.

Step 1: I do an API query to get the list of dataset names and another to get the list of field names in the tephrabase dataset

In [9]:
# Collect the tephrabase dictioary
tephrabase_dataset = get_dataset_by_name('tephrabase')
# Extract the fields from the dictionary directly
tephrabase_dataset_fields = tephrabase_dataset.get('fields', [])
# Construct the DataFrame to inspect the fields
tephrabase_dataset_fields_df = pd.DataFrame(tephrabase_dataset_fields)

Step 2: I can see the earliest date and latest date fields in the tephrabase dataset. I construct an API query to get the tephrabase data into a dataframe.

In [10]:
# Using the `id` within the dataset dictionary, collect all the features
# Again, we'll use our handy function for this
tephrabase_features_df = get_features_by_dataset_id(tephrabase_dataset.get('id'))

Step 3: I filter my dataframe to select rows where df.mydataset = df.loc[tephrabase(latest date - earliest date)<100] to get sequences of less than 100 years

In [11]:
# Apply the conditions that we only need less than 100 year span
# From the fields above we see it's `latestyear` and `earliestyear`
tephrabase_less_than_100 = tephrabase_features_df[
    (tephrabase_features_df['latestyear'] - tephrabase_features_df['earliestyear']) < 100
]
tephrabase_less_than_100

,sitenumber,sitename,sitelink,profilenumber,profilename,earliestyear,latestyear,tephra,_id
1,9,Orrastaðir,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,71,Sv11 (Pit 11),1104,1104,"[{'tephra_tephraname': 'Hekla 1104 Tephra', 't...",5ff5f9f1e7ac17001c040f2b
10,95,Kulunes,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,82,Profile 1,1300,1300,"[{'tephra_tephraname': 'Hekla 1300 Tephra', 't...",5ff5f9f1e7ac17001c040f34
13,19,Snidanes,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,192,Pit 1,1875,1875,"[{'tephra_tephraname': 'Askja 1875 Tephra', 't...",5ff5f9f1e7ac17001c040f37
14,213,Paela,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,193,Pit 1,1510,1510,"[{'tephra_tephraname': 'Hekla 1510 Tephra', 't...",5ff5f9f1e7ac17001c040f38
90,1373,Sol131,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,1238,Sol131A,1918,1986,"[{'tephra_tephraname': 'Hekla 1947 Tephra', 't...",5ff5f9f1e7ac17001c040f84
91,1373,Sol131,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,1239,Sol131B,1918,1986,"[{'tephra_tephraname': 'Hekla 1947 Tephra', 't...",5ff5f9f1e7ac17001c040f85
92,1373,Sol131,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,1240,Sol131C,1918,1986,"[{'tephra_tephraname': 'Hekla 1947 Tephra', 't...",5ff5f9f1e7ac17001c040f86
137,1424,Seljaland - Transect PitA,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,1295,Sjl-T PitA,1947,1998,"[{'tephra_tephraname': 'Hekla 1947 Tephra', 't...",5ff5f9f1e7ac17001c040fb3
138,1425,Seljaland - Transect PitB,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,1296,Sjl-T PitB,1947,1998,"[{'tephra_tephraname': 'Hekla 1947 Tephra', 't...",5ff5f9f1e7ac17001c040fb4
139,1426,Seljaland - Transect PitC,https://www.tephrabase.org/cgi-bin/tbase_ice_t...,1297,Sjl-T PitC,1947,1998,"[{'tephra_tephraname': 'Hekla 1947 Tephra', 't...",5ff5f9f1e7ac17001c040fb5


Step 4: Assuming this gives me a reasonable set of sites, I do another API query to see what concepts are mapped to these sites.

In [12]:
# Collect the unique list of balanced feature _id values for the API
unique_feature_ids = set(tephrabase_less_than_100['_id'])

# To get concepts, we must first travel through the combinators
# Collect combinators
combinators = get_combinators_from_features(unique_feature_ids)

# For this we want the `related` and `contextual` combinators
related = combinators.get('related')
contextual = combinators.get('contextual')

In [13]:
# Use the combinator ids to collect concepts using another handy function
related_df = make_concepts_request(related)
contextual_df = make_concepts_request(contextual)

In [14]:
related_df

,group,related,contextual,_id,title,name,updatedAt,url,id,description,citation,createdAt,__v
0,physical_landscape,"[{'group': 'activities', 'related': ['5fad8ae3...","[{'group': None, 'related': ['5fc9da3e32cd3c5d...",5f430a74836c1e48136b4f77,built environment,built_environment_1,2021-01-07T15:16:26.785Z,https://github.com/castuofa/dataarc-source/blo...,5f430a74836c1e48136b4f77,NaN,NaN,NaN,NaN
1,physical_landscape,"[{'group': 'physical_processes', 'related': ['...","[{'group': 'physical_processes', 'related': ['...",5f430a75836c1e48136b4fba,glacier,glacier_1,2021-01-07T15:31:49.622Z,https://github.com/castuofa/dataarc-source/blo...,5f430a75836c1e48136b4fba,NaN,NaN,NaN,NaN
2,physical_landscape,"[{'group': 'activities', 'related': ['5f430a76...","[{'group': 'actors', 'related': ['5fad8ac12e60...",5f430a76836c1e48136b4fcf,house,house_1,2021-01-07T15:35:49.266Z,https://github.com/castuofa/dataarc-source/blo...,5f430a76836c1e48136b4fcf,NaN,NaN,NaN,NaN
3,physical_processes,"[{'group': 'physical_processes', 'related': ['...","[{'group': None, 'related': ['5fad8b7f2e604b08...",5f430a76836c1e48136b4fe0,jokulhlaups,jokulhlaups_1,2021-01-07T15:42:34.115Z,https://github.com/castuofa/dataarc-source/blo...,5f430a76836c1e48136b4fe0,NaN,NaN,NaN,NaN
4,physical_processes,"[{'group': 'activities', 'related': ['5fad8ae3...","[{'group': None, 'related': ['5fc9da3e32cd3c5d...",5f430a76836c1e48136b4fe3,land degredation,land_degredation_1,2021-01-07T15:37:56.182Z,https://github.com/castuofa/dataarc-source/blo...,5f430a76836c1e48136b4fe3,NaN,NaN,NaN,NaN
5,activities,"[{'group': 'activities', 'related': ['5fad8ae3...","[{'group': None, 'related': ['5fc9da3e32cd3c5d...",5f430a76836c1e48136b4fe4,land management,land_management_1,2021-01-07T15:38:52.173Z,https://github.com/castuofa/dataarc-source/blo...,5f430a76836c1e48136b4fe4,NaN,NaN,NaN,NaN
6,actors,"[{'group': 'ideas', 'related': ['5fad8b7f2e604...","[{'group': None, 'related': ['5fad8ab82e604b08...",5f430a76836c1e48136b4fe6,lava,lava,2020-12-04T06:49:45.275Z,NaN,5f430a76836c1e48136b4fe6,NaN,NaN,NaN,NaN
7,ideas,"[{'group': 'ideas', 'related': ['5f430a77836c1...","[{'group': 'physical_processes', 'related': ['...",5f430a76836c1e48136b4fea,little ice age,little_ice_age_1,2021-01-07T15:43:51.711Z,https://github.com/castuofa/dataarc-source/blo...,5f430a76836c1e48136b4fea,NaN,NaN,NaN,NaN
8,actors,"[{'group': None, 'related': ['5f430a76836c1e48...","[{'group': 'actors', 'related': ['5fad8ac12e60...",5f430a76836c1e48136b4feb,live human,live_human,2020-12-04T06:49:45.059Z,NaN,5f430a76836c1e48136b4feb,NaN,NaN,NaN,NaN
9,ideas,"[{'group': 'ideas', 'related': ['5f430a77836c1...","[{'group': 'ideas', 'related': ['5f430a77836c1...",5f430a77836c1e48136b4ffb,modeled climatic systems,modeled_climatic_systems,2020-12-04T06:49:45.172Z,NaN,5f430a77836c1e48136b4ffb,NaN,NaN,NaN,NaN


In [15]:
contextual_df

,group,related,contextual,_id,title,name,updatedAt,url,id,description,citation,createdAt,__v
0,physical_landscape,"[{'group': 'activities', 'related': ['5fc9da58...","[{'group': 'activities', 'related': ['5fad8ae3...",5f430a74836c1e48136b4f83,charcoal pit,charcoal_pit_1,2021-01-07T15:20:53.716Z,https://github.com/castuofa/dataarc-source/blo...,5f430a74836c1e48136b4f83,NaN,NaN,NaN,NaN
1,physical_landscape,"[{'group': 'activities', 'related': ['5fc9da58...","[{'group': 'activities', 'related': ['5fad8ae3...",5f430a74836c1e48136b4f98,disturbed or arable,disturbed_or_arable_1,2021-01-07T15:23:34.870Z,https://github.com/castuofa/dataarc-source/blo...,5f430a74836c1e48136b4f98,NaN,NaN,NaN,NaN
2,physical_landscape,"[{'group': 'activities', 'related': ['5fc9da58...","[{'group': 'activities', 'related': ['5fad8ae3...",5f430a74836c1e48136b4f9d,dry dead wood,dry_dead_wood_1,2021-01-07T15:24:07.399Z,https://github.com/castuofa/dataarc-source/blo...,5f430a74836c1e48136b4f9d,NaN,NaN,NaN,NaN
3,actors,"[{'group': 'events', 'related': ['5f430a76836c...","[{'group': 'actors', 'related': ['5f430a78836c...",5f430a75836c1e48136b4fa6,family,family_1,2021-01-07T15:27:44.267Z,https://github.com/castuofa/dataarc-source/blo...,5f430a75836c1e48136b4fa6,NaN,NaN,NaN,NaN
4,community_places,"[{'group': 'actors', 'related': ['5f430a74836c...","[{'group': 'events', 'related': ['5f430a76836c...",5f430a75836c1e48136b4fa7,farm,farm_1,2021-01-07T15:26:02.142Z,https://github.com/castuofa/dataarc-source/blo...,5f430a75836c1e48136b4fa7,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,None,"[{'group': 'ideas', 'related': ['5fad8ae32e604...","[{'group': None, 'related': ['5fc9da3e32cd3c5d...",5fad8ae32e604b08b11c1dc4,events,events_1,2021-01-07T17:09:12.353Z,https://github.com/castuofa/dataarc-source/blo...,5fad8ae32e604b08b11c1dc4,,,2020-11-12T19:20:03.870Z,0.0
58,None,"[{'group': None, 'related': ['5fad8b7f2e604b08...","[{'group': None, 'related': ['5fad8ab82e604b08...",5fad8b7f2e604b08b11c1dc8,imaginary landscape,imaginary_landscape_1,2021-01-07T17:09:41.847Z,https://github.com/castuofa/dataarc-source/blo...,5fad8b7f2e604b08b11c1dc8,,,2020-11-12T19:22:39.263Z,0.0
59,None,"[{'group': 'ideas', 'related': ['5fad8c092e604...","[{'group': None, 'related': ['5f430a78836c1e48...",5fad8c092e604b08b11c1dcc,physical processes,physical_processes_1,2021-01-07T17:09:59.601Z,https://github.com/castuofa/dataarc-source/blo...,5fad8c092e604b08b11c1dcc,NaN,NaN,2020-11-12T19:24:58.000Z,0.0
60,ideas,"[{'group': None, 'related': ['5fc9da3e32cd3c5d...","[{'group': 'ideas', 'related': ['5fad8ae32e604...",5fc9da3e32cd3c5d8ea39b6d,abandonment,abandonment_1,2021-01-07T17:10:19.541Z,https://github.com/castuofa/dataarc-source/blo...,5fc9da3e32cd3c5d8ea39b6d,,,2020-12-04T06:42:06.267Z,0.0


Step 5: I go back to the UI and apply a spatial filter to look at the specific sites I've identified here and see what other data is related to them AND is mapped to the same concepts (by filtering by concept).